In [32]:
import openai
import langchain
from dotenv import load_dotenv,find_dotenv

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
#from langchain.llms.openai import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import JSONLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import json
from pathlib import Path
from pprint import pprint
import pinecone
from tqdm.notebook import tqdm # need this to stop warning

import os
import pinecone # pip3 install "pinecone-client[grpc]"
from langchain.vectorstores import Pinecone

from pprint import pprint # pretty print

# load environmnet variables
load_dotenv('.env')

True

In [33]:
### Load only the syllabus summaerized text files

In [34]:
directory = './pdf_word/'

def load_docs(directory):
  loader = DirectoryLoader(directory,silent_errors=True)
  documents = loader.load()
  return documents

documents_text = load_docs(directory)
len(documents_text)

284

In [35]:
def split_docs(documents,chunk_size=950,chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

In [36]:
docs_docx = split_docs(documents_text)

In [38]:
docs_docx[0].page_content

'Final Exam (30%): The final exam will be based on course content covered throughout the course. Professor Terry A. Boyd is teaching Human Resource Management (MAN3301) at the Muma College of Business, University of South Florida in the Spring 2023 semester. The course will cover topics such as the mission/values of the organization and its relationship to HRM, staffing, employee training/development, compensation, and societal responsibility of the organization. Students will be assessed through participation/blogs (20%), quizzes (30%), mid-term exam (20%) and final exam (30%). The course will help students to become fully acquainted with the academic concepts of HRM and apply them to day to day situations at the workplace.This document is about the Human Resource Management Course at the MUMA College of Business, taught by Professor X. The course topics include a mid-term, final exam, Tableau Assignment, course policies on grades and'

In [39]:
len(docs_docx)

1158

In [40]:
## embedding
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002")

In [41]:
### Loading in to pinecone:

In [42]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENV")

In [43]:
# create an index
# NOTE: For free version, you can only create one index. If you want to create a new index, you need to delete the old one first.

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

#pinecone.delete_index("chatbot")

pinecone.create_index("chatbot", dimension=1536) # 1536 is openai ada embedding dimension

In [44]:
index_name = "chatbot"

In [45]:
search = Pinecone.from_documents(docs_docx, embeddings, index_name=index_name)

In [46]:
### Documents from the web

In [47]:
directory = './All_Texts/'

def load_docs(directory):
  loader = DirectoryLoader(directory,silent_errors=True)
  documents = loader.load()
  return documents

documents_text = load_docs(directory)
len(documents_text)

21

In [48]:
def split_docs(documents,chunk_size=500,chunk_overlap=40):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

In [49]:
docs_docx_1 = split_docs(documents_text)

In [50]:
len(docs_docx_1)

516

In [51]:
search = Pinecone.from_documents(docs_docx_1, embeddings, index_name=index_name)

In [52]:
############################### Checking the accuracy

In [53]:
from langchain.vectorstores import Pinecone
text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

In [54]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [55]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [56]:
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [57]:
query = "What is the appplication deadline for fall?"

vectorstore.similarity_search_with_score(query, k=2)

[(Document(page_content='Science this semester. Important dates to remember include the Drop/Add Deadline on Jan 14, Fall Graduation Application Deadline on Feb 7, Withdrawal Deadline on March 26, MLK Day Observed Holiday on Jan 17, Spring Break from March 14-20, Test Free Week, and Final Examinations from Apr 23-29 and Apr 30-May 5. Due to Covid, Professor Smith will deliver the course in-person and provide a flexible component for students who are asked to isolate or quarantine, or feel unable to attend a class in-person for health reasons.', metadata={'source': 'pdf_word\\cleaned_2023 Spring_MAN 4402-902_Employment Laws_Papp,Timothy.docx'}),
  0.83433044),
 (Document(page_content='students. These resources provide services such as counseling, tutoring, writing consultations, victim advocacy, and medical services. Appointments are recommended but not required. Contact information for each resource is available online. Important dates to remember include the semester start date, drop/

In [58]:
query = "What are the core courses for the bais program"

vectorstore.similarity_search_with_score(query, k=3)

[(Document(page_content='Course Summary This course is an opportunity for Master of Science Business Analytics and Information Systems (MS BAIS) to gain insights into opportunities and challenges of decision-making in contemporary business situations.The instructor is Dr. Priya Dozier and the course topics include the development of enterprise transaction processing applications, relational database management, database sharing, CASE methodology and project management techniques.Students will be required to purchase a course pack of simulations via HBR Publishing and the final grades will be calculated according to the points scale. This course is Enterprise Information Systems Management, taught by Dr. Priya Dozier. It is a capstone course for Master of Science Business Analytics and Information Systems (MS BAIS) students, and it bridges the gap between technology systems/projects and strategic/tactical business decisions. Topics include the development of', metadata={'source': 'pdf_w

In [59]:
query = "What are the oncampus housing options?"

vectorstore.similarity_search_with_score(query, k=2)

[(Document(page_content='Center, Student Outreach & Support, Ombudsman, Student Conduct & Ethical Development, Student Health Services, and the Dean of Students. It also provides off-campus confidential and non-confidential resources. Finally, it provides a disclaimer about the Campus Free Expression Act.', metadata={'source': 'pdf_word\\cleaned_ISM 6155.360 - Dozier.docx'}),
  0.814621091),
 (Document(page_content='Accommodations. Confidential resources such as the Center for Victim Advocacy and Violence Prevention, the Counseling Center and Student Health Services are also available.Professor: QMB6358.040S23.25239 Data Analytics for Business. Course topics include: First Day Attendance Quiz, Getting Started Quiz, M1 Quiz, M1 Discussion, M2 Discussion, M2 Quiz, Kaggle Project Discussion, Kaggle Project Quiz 1, M3 Discussion, M3 Quiz, Kaggle Project Discussion, Tableau Dashboard Discussion, Tableau Dashboard Submission, Kaggle Project Quiz 2, Kaggle Project Discussion, Kaggle Project Q

In [60]:
query = "Project management course syllabus "

vectorstore.similarity_search_with_score(query, k=2)

[(Document(page_content='Course Topics: This course provides a practical understanding of Project Management (PM) with particular focus on its application in business organizations. It also focuses on understanding Agile Project Management (APM) and introduces Scrum as a popular Agile project management approach. The course further explores the important role of leadership together with its psycho-social aspects in enabling successful Agile adoption in the organization. Students will also use a relevant tool (Trello) to give them a taste of use of tools in online collaboration in an Agile project.The course is called "Agile Project Management" and is taught by Professor B Unhelkar. The course topics include recognizing project management basics including risks, scope, time, budget, quality and change management, recognizing the importance of agility in projects and organizational culture, demonstrating the use of agile techniques used in practical projects', metadata={'source': 'pdf_wo

In [61]:
query = "Tim Smith "
vectorstore.similarity_search_with_score(query, k=3)

[(Document(page_content='Each module contains lecture slides, assignments, quizzes, and other materials. Professor: Dr. Tim Smith', metadata={'source': 'pdf_word\\cleaned_2023 Spring_ISM 6251-002-Data Science Programming_Smith, Tim.docx'}),
  0.82448),
 (Document(page_content='Each module contains lecture slides, assignments, quizzes, and other materials.You will be expected to complete the assignments and quizzes for each module. Professor: Dr. Tim Smith', metadata={'source': 'pdf_word\\cleaned_2023 Spring_ISM 6251-003-Data Science Programming_Smith, Tim.docx'}),
  0.802567959),
 (Document(page_content="What Nature Can Teach Us about Sustainable Business, and Interface Carpets and Sustainability Metrics. The final assignment is to prepare a 5 minute video summary of a randomly assigned chapter from Anderson's Business Lessons. The exam is an essay question due before 1:00 pm on April 21st and there are two reading days with no class.", metadata={'source': 'pdf_word\\cleaned_2023 Sprin

In [62]:
query = "Business realted prerequisities for the MS BAIS program"

vectorstore.similarity_search_with_score(query, k=3)

[(Document(page_content='Course Summary This course is an opportunity for Master of Science Business Analytics and Information Systems (MS BAIS) to gain insights into opportunities and challenges of decision-making in contemporary business situations.The instructor is Dr. Priya Dozier and the course topics include development of enterprise transaction processing applications, relational database management, database sharing, CASE methodology and project management techniques.Students will be required to purchase a course pack via HBR Publishing and will be graded according to the grading scale. This course is Enterprise Information Systems Management, taught by Dr. Priya Dozier. It is a capstone course for Master of Science Business Analytics and Information Systems (MS BAIS) students, and covers topics such as development of enterprise transaction processing applications, relational database management, database sharing, CASE methodology and project management', metadata={'source': 'p